<a href="https://colab.research.google.com/github/LeandroCoelhos/estudos_datascience/blob/main/LGBM/LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Vantagens do LGBM

Otimização em velocidade e uso de memória

Muitas ferramentas de boosting usam algoritmos baseados em pré-ordenação [2, 3] (por exemplo, algoritmo padrão em xgboost) para aprendizado de árvores de decisão. É uma solução simples, mas não fácil de otimizar.

O LightGBM usa algoritmos baseados em histograma [4, 5, 6] , que agrupam valores de recursos contínuos (atributos) em compartimentos discretos. Isso acelera o treinamento e reduz o uso de memória. As vantagens dos algoritmos baseados em histogramas incluem o seguinte:

    Custo reduzido de cálculo do ganho para cada divisão

        Algoritmos baseados em pré-ordenação têm complexidade de tempoO(#data)

        Calcular o histograma tem complexidade de tempo O(#data), mas isso envolve apenas uma operação de soma rápida. Uma vez que o histograma é construído, um algoritmo baseado em histograma tem complexidade O(#bins)de tempo e #binsé muito menor que #data.

    Use a subtração do histograma para acelerar ainda mais

        Para obter os histogramas de uma folha em uma árvore binária, use a subtração do histograma de seu pai e seu vizinho

        Portanto, ele precisa construir histogramas para apenas uma folha (com menor #dataque sua vizinha). Ele então pode obter histogramas de seu vizinho por subtração de histograma com pequeno custo ( O(#bins))

    Reduza o uso de memória

        Substitui valores contínuos por compartimentos discretos. Se #binsfor pequeno, pode usar o tipo de dados pequeno, por exemplo, uint8_t, para armazenar dados de treinamento

        Não há necessidade de armazenar informações adicionais para valores de recursos de pré-classificação

    Reduza o custo de comunicação para aprendizado distribuído



Suporte a variaveis categóricas sem fazer one hot
* categorical_feature

# Parâmetros
num_leaves, default =31, type = int, aliases:num_leaf,max_leaves,max_leaf,max_leaf_nodes, restrições:1 < num_leaves <= 131072

    número máximo de folhas em uma árvore
<hr>

tree_learner , default =serial, type = enum, opções:serial,feature,data,voting, aliases:tree,tree_type,tree_learner_type

    serial, aprendizado de árvore de máquina única

    feature, recurso de aprendizado de árvore paralela, aliases:feature_parallel

    data, aprendiz de árvore paralela de dados, aliases:data_parallel

    voting, aluno de árvore paralela de votação, aliases:voting_parallel

    consulte o Guia de Aprendizagem Distribuída para obter mais detalhes - https://lightgbm-readthedocs-io.translate.goog/en/v3.3.2/Parallel-Learning-Guide.html?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt-BR&_x_tr_pto=sc
<hr>

num_threads 🔗︎ , default =0, type = int, aliases:num_thread,nthread,nthreads,n_jobs

    número de threads para LightGBM

    0significa o número padrão de threads no OpenMP

    para a melhor velocidade, defina isso para o número de núcleos de CPU reais , não o número de threads (a maioria das CPUs usa hyper-threading para gerar 2 threads por núcleo de CPU)

    não defina muito grande se seu conjunto de dados for pequeno (por exemplo, não use 64 threads para um conjunto de dados com 10.000 linhas)

    esteja ciente de que um gerenciador de tarefas ou qualquer ferramenta de monitoramento de CPU semelhante pode relatar que os núcleos não estão sendo totalmente utilizados. Isto é normal

    para aprendizado distribuído, não use todos os núcleos da CPU, pois isso causará um desempenho ruim para a comunicação da rede

    Nota : por favor , não altere isso durante o treinamento, especialmente ao executar vários trabalhos simultaneamente por pacotes externos, caso contrário, poderá causar erros indesejáveis



device_type 🔗︎ , default =cpu, type = enum, opções:cpu,gpu,cuda, aliases:device

    dispositivo para o aprendizado em árvore, você pode usar a GPU para obter o aprendizado mais rápido

    Nota : recomenda-se usar o menor max_bin(por exemplo, 63) para obter a melhor velocidade

    Nota : para a velocidade mais rápida, a GPU usa o ponto flutuante de 32 bits para resumir por padrão, portanto, isso pode afetar a precisão de algumas tarefas. Você pode configurar gpu_use_dp=truepara habilitar o ponto flutuante de 64 bits, mas isso retardará o treinamento

    Nota : consulte o Guia de instalação para construir LightGBM com suporte a GPU

<hr>
seed 🔗︎ , default =None, type = int, aliases:random_seed,random_state

    esta semente é usada para gerar outras sementes, por exemplo data_random_seed, feature_fraction_seed, etc.

    por padrão, esta semente não é usada em favor dos valores padrão de outras sementes

    esta semente tem prioridade menor em comparação com outras sementes, o que significa que ela será substituída, se você definir outras sementes explicitamente









# Hiperparãmetros de Controle de Aprendizagem

<hr>
max_depth 🔗︎ , padrão =-1, tipo = int

    limitar a profundidade máxima para o modelo de árvore. Isso é usado para lidar com o ajuste excessivo quando #dataé pequeno. A árvore ainda cresce em folha

    <= 0significa sem limite


<hr>
min_data_in_leaf 🔗︎ , default =20, type = int, aliases:min_data_per_leaf,min_data,min_child_samples,min_samples_leaf, restrições:min_data_in_leaf >= 0

    número mínimo de dados em uma folha. Pode ser usado para lidar com sobre-ajuste

    Nota : esta é uma aproximação baseada no Hessian, então ocasionalmente você pode observar divisões que produzem nós folha que têm menos do que este número de observações


<hr>
<b>Desbalanceamento</b>

pos_bagging_fraction 🔗︎ , default =1.0, type = double, aliases:pos_sub_row,pos_subsample,pos_bagging, restrições:0.0 < pos_bagging_fraction <= 1.0

    usado apenas em binaryaplicações

    usado para o problema de classificação binária desequilibrada, amostrará aleatoriamente amostras positivas em ensacamento#pos_samples * pos_bagging_fraction

    deve ser usado em conjunto comneg_bagging_fraction

    defina isso 1.0para desabilitar

    Nota : para habilitar isso, você precisa definir bagging_freqe neg_bagging_fractiontambém

    Nota : se ambos pos_bagging_fractione neg_bagging_fractionestiverem definidos como 1.0, o ensacamento balanceado será desabilitado

    Nota : se o ensacamento balanceado estiver habilitado, bagging_fractionserá ignorado

<hr>
neg_bagging_fraction 🔗︎ , default =1.0, type = double, aliases:neg_sub_row,neg_subsample,neg_bagging, restrições:0.0 < neg_bagging_fraction <= 1.0

    usado apenas em binaryaplicações

    usado para o problema de classificação binária desequilibrada, amostrará aleatoriamente amostras negativas no ensacamento#neg_samples * neg_bagging_fraction

    deve ser usado em conjunto compos_bagging_fraction

    defina isso 1.0para desabilitar

    Nota : para habilitar isso, você precisa definir bagging_freqe pos_bagging_fractiontambém

    Nota : se ambos pos_bagging_fractione neg_bagging_fractionestiverem definidos como 1.0, o ensacamento balanceado será desabilitado

    Nota : se o ensacamento balanceado estiver habilitado, bagging_fractionserá ignorado


<hr>
bagging_freq 🔗︎ , default =0, type = int, aliases:subsample_freq

    frequência de ensacamento

    0significa desabilitar o ensacamento; ksignifica realizar o ensacamento a cada kiteração. A cada k-th iteração, o LightGBM selecionará aleatoriamente os dados a serem usados ​​nas próximas iteraçõesbagging_fraction * 100 %k

    Nota : para habilitar o ensacamento, bagging_fractiondeve ser configurado para valor menor que 1.0também


<hr>
<hr>
early_stopping_round 🔗︎ , default =0, type = int, aliases:early_stopping_rounds,early_stopping,n_iter_no_change

    irá parar de treinar se uma métrica de um dado de validação não melhorar nas últimas early_stopping_roundrodadas

    <= 0significa desabilitar

    pode ser usado para acelerar o treinamento


<hr>
extra_trees 🔗︎ , default =false, type = bool, aliases:extra_tree

    use árvores extremamente aleatórias

    se definido como true, ao avaliar as divisões de nós, o LightGBM verificará apenas um limite escolhido aleatoriamente para cada recurso

    pode ser usado para acelerar o treinamento

    pode ser usado para lidar com excesso de ajuste

<hr>
path_smooth 🔗︎ , default =0, type = double, restrições:path_smooth >=  0.0

    controla a suavização aplicada aos nós da árvore

    ajuda a evitar overfitting em folhas com poucas amostras

    se definido como zero, nenhuma suavização é aplicada

    se então deve ser pelo menospath_smooth > 0min_data_in_leaf2

    valores maiores dão uma regularização mais forte

        o peso de cada nó é , onde é o número de amostras no nó, é o peso ideal do nó para minimizar a perda (aproximadamente ), e é o peso do nó pai(n / path_smooth) * w + w_p / (n / path_smooth + 1)nw-sum_gradients / sum_hessiansw_p

        observe que a própria saída pai w_ptem suavização aplicada, a menos que seja o nó raiz, para que o efeito de suavização se acumule com a profundidade da árvore





# Parâmetros de E/S

<hr>
max_bin 🔗︎ , default =255, type = int, aliases:max_bins, restrições:max_bin > 1

    número máximo de compartimentos nos quais os valores de recurso serão agrupados

    um pequeno número de caixas pode reduzir a precisão do treinamento, mas pode aumentar a potência geral (lidar com o ajuste excessivo)

    LightGBM irá comprimir automaticamente a memória de acordo com max_bin. Por exemplo, LightGBM usará uint8_tpara valor de recurso semax_bin=255
<hr>
zero_as_missing 🔗︎ , default =false, type = bool

    defina isso para truetratar todos os zeros como valores ausentes (incluindo os valores não mostrados em LibSVM / matrizes esparsas)

    defina isso para falseusar napara representar valores ausentes
<hr>
two_round 🔗︎ , default =false, type = bool, aliases:two_round_loading,use_two_round_loading

    defina isso para truese o arquivo de dados for muito grande para caber na memória

    por padrão, o LightGBM mapeará o arquivo de dados para a memória e carregará recursos da memória. Isso fornecerá uma velocidade de carregamento de dados mais rápida, mas pode causar um erro de falta de memória quando o arquivo de dados for muito grande

    Nota : funciona apenas no caso de carregar dados diretamente do arquivo de texto
<hr>
header 🔗︎ , default =false, type = bool, aliases:has_header

    defina isso para truese os dados de entrada tiverem cabeçalho

    Nota : funciona apenas no caso de carregar dados diretamente do arquivo de texto
<hr>
group_column 🔗︎ , default ="", type = int ou string, aliases:group,group_id,query_column,query,query_id

    usado para especificar a coluna de ID de consulta/grupo

    use número para índice, por exemplo, query=0significa que column_0 é o id da consulta

    adicione um prefixo name:para o nome da coluna, por exemploquery=name:query_id

    Nota : funciona apenas no caso de carregar dados diretamente do arquivo de texto

    Nota : os dados devem ser agrupados por query_id, para mais informações, consulte Query Data

    Nota : o índice começa 0e não conta a coluna do rótulo quando o tipo de passagem é int, por exemplo, quando o rótulo é column_0 e query_id é column_1, o parâmetro correto équery=0
<hr>
ignore_column 🔗︎ , default ="", type = multi-int ou string, aliases:ignore_feature,blacklist

    usado para especificar algumas colunas ignoradas no treinamento

    use número para índice, por exemplo, ignore_column=0,1,2significa que column_0, column_1 e column_2 serão ignorados

    adicione um prefixo name:para o nome da coluna, por exemplo, ignore_column=name:c1,c2,c3significa que c1, c2 e c3 serão ignorados

    Nota : funciona apenas no caso de carregar dados diretamente do arquivo de texto

    Nota : o índice começa 0e não conta a coluna do rótulo quando o tipo de passagem éint

    Nota : apesar do fato de que as colunas especificadas serão completamente ignoradas durante o treinamento, elas ainda devem ter um formato válido permitindo que o LightGBM carregue o arquivo com sucesso
<hr>
categorical_feature 🔗︎ , default ="", type = multi-int ou string, aliases:cat_feature,categorical_column,cat_column,categorical_features

    usado para especificar recursos categóricos

    use número para índice, por exemplo, categorical_feature=0,1,2significa que column_0, column_1 e column_2 são recursos categóricos

    adicione um prefixo name:para o nome da coluna, por exemplo, categorical_feature=name:c1,c2,c3significa que c1, c2 e c3 são recursos categóricos

    Nota : suporta apenas categórico com inttipo (não aplicável para dados representados como pandas DataFrame no pacote Python)

    Nota : o índice começa 0e não conta a coluna do rótulo quando o tipo de passagem éint

    Nota : todos os valores devem ser menores que Int32.MaxValue(2147483647)

    Nota : usar valores grandes pode consumir memória. A regra de decisão da árvore funciona melhor quando os recursos categóricos são apresentados por inteiros consecutivos começando em zero

    Nota : todos os valores negativos serão tratados como valores ausentes

    Nota : a saída não pode ser restrita monotonicamente em relação a um recurso categórico
<hr>
<hr>
<hr>


# Praâmetros Objetivos

<b> Desbalanceados </b>


<hr>
is_unbalance 🔗︎ , default =false, type = bool, aliases:unbalance,unbalanced_sets

    usado apenas em binarye multiclassovaaplicações

    defina isso para truese os dados de treinamento estiverem desequilibrados

    Observação : embora habilitar isso deva aumentar a métrica de desempenho geral do seu modelo, também resultará em estimativas ruins das probabilidades de classe individual

    Nota : este parâmetro não pode ser usado ao mesmo tempo com scale_pos_weight, escolha apenas um deles
<hr>
scale_pos_weight 🔗︎ , default =1.0, type = double, restrições:scale_pos_weight > 0.0

    usado apenas em binarye multiclassovaaplicações

    peso dos rótulos com classe positiva

    Observação : embora habilitar isso deva aumentar a métrica de desempenho geral do seu modelo, também resultará em estimativas ruins das probabilidades de classe individual

    Nota : este parâmetro não pode ser usado ao mesmo tempo com is_unbalance, escolha apenas um deles
<hr>
<hr>
<hr>
<hr>


# Parãmetros de Métricas


<hr>
metric 🔗︎ , default ="", type = multi-enum, aliases:metrics,metric_types

    métrica(s) a ser(em) avaliada(s) no(s) conjunto(s) de avaliação

        ""(string vazia ou não especificada) significa que a métrica correspondente à especificada objectiveserá usada (isso é possível apenas para funções objetivas predefinidas, caso contrário, nenhuma métrica de avaliação será adicionada)

        "None"(string, não um Nonevalor) significa que nenhuma métrica será registrada, aliases: na, null,custom

        l1, perda absoluta, aliases: mean_absolute_error, mae,regression_l1

        l2, perda quadrada, aliases: mean_squared_error, mse, regression_l2,regression

        rmse, perda de raiz quadrada, aliases: root_mean_squared_error,l2_root

        quantile, regressão quantílica

        mape, perda MAPE , aliases:mean_absolute_percentage_error

        huber, Perda de Huber

        fair, Perda justa

        poisson, probabilidade logarítmica negativa para regressão de Poisson

        gamma, probabilidade logarítmica negativa para regressão gama

        gamma_deviance, desvio residual para regressão gama

        tweedie, probabilidade logarítmica negativa para regressão de Tweedie

        ndcg, NDCG , aliases: lambdarank, rank_xendcg, xendcg, xe_ndcg, xe_ndcg_mart,xendcg_mart

        map, MAP , aliases:mean_average_precision

        auc, AUC

        average_precision, pontuação de precisão média

        binary_logloss, perda de log , aliases:binary

        binary_error, para uma amostra: 0para classificação correta, 1para classificação de erros

        auc_mu, AUC-mu

        multi_logloss, perda de log para classificação multiclasse, aliases: multiclass, softmax, multiclassova, multiclass_ova, ova,ovr

        multi_error, taxa de erro para classificação multiclasse

        cross_entropy, entropia cruzada (com pesos lineares opcionais), aliases:xentropy

        cross_entropy_lambda, entropia cruzada “ponderada por intensidade”, aliases:xentlambda

        kullback_leibler, divergência de Kullback-Leibler , aliases:kldiv



# Ajuste de Árvore

<hr>
Para obter bons resultados usando uma árvore foliar, estes são alguns parâmetros importantes:

num_leaves. Este é o principal parâmetro para controlar a complexidade do modelo de árvore. Teoricamente, podemos definir para obter o mesmo número de folhas que a árvore em profundidade. No entanto, essa conversão simples não é boa na prática. A razão é que uma árvore de folha é tipicamente muito mais profunda do que uma árvore de profundidade para um número fixo de folhas. A profundidade irrestrita pode induzir o ajuste excessivo. Assim, ao tentar afinar o , devemos deixá-lo menor que . Por exemplo, quando a árvore de profundidade pode obter uma boa precisão, mas definir para pode causar um ajuste excessivo e defini-la para ou pode obter uma precisão melhor do que a de profundidade.num_leaves = 2^(max_depth)num_leaves2^(max_depth)max_depth=7num_leaves1277080


min_data_in_leaf. Este é um parâmetro muito importante para evitar o ajuste excessivo em uma árvore foliar. Seu valor ideal depende do número de amostras de treinamento e num_leaves. Defini-lo para um valor alto pode evitar o crescimento muito profundo de uma árvore, mas pode causar um ajuste insuficiente. Na prática, configurá-lo para centenas ou milhares é suficiente para um grande conjunto de dados.

max_depth. Você também pode usar max_depthpara limitar explicitamente a profundidade da árvore.
